# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data = pd.read_csv("../WeatherPy/cities.csv")
cities_df = pd.DataFrame(cities_data)
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kapaa,90,US,1612801077,82,22.0752,-159.3190,62.60,9.22
1,1,Port-Cartier,100,CA,1612801077,74,50.0334,-66.8654,25.00,9.22
2,2,Kungurtug,24,RU,1612801077,86,50.5994,97.5228,10.33,3.60
3,3,Tuktoyaktuk,1,CA,1612801077,74,69.4541,-133.0374,-40.00,6.38
4,4,Aklavik,100,CA,1612801077,67,68.2191,-135.0107,-36.40,3.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring gmaps
gmaps.configure(api_key=g_key)

In [4]:
# creating the locations and weight from the cities_df
locations = cities_df[["Lat", "Lng"]].astype(float)
my_weights = cities_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure(center=(30,20), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=my_weights, dissipating=False, max_intensity=my_weights.max(),
                                point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Ideal conditions: Temp between 70-80, wind speed < 5, cloudiness = 0 
# store into a variable called hotel_df
hotel_df = pd.DataFrame()
hotel_df = cities_df.loc[(cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 90) 
                                    & (cities_df["Wind Speed"] < 5) & (cities_df["Cloudiness"] == 0)] 

hotel_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
59,59,Champerico,0,GT,1612801085,66,14.3000,-91.9167,86.00,3.04
83,83,Castro,0,BR,1612801087,51,-24.7911,-50.0119,74.95,4.65
124,124,Safaga,0,EG,1612801093,33,26.7292,33.9365,75.20,3.44
207,207,Coyhaique,0,CL,1612801103,31,-45.5752,-72.0662,75.20,3.44
257,257,Ancud,0,CL,1612801109,44,-41.8697,-73.8203,81.00,4.61
303,303,Jerusalem,0,IL,1612800974,56,31.7690,35.2163,71.01,3.44
510,510,Umluj,0,SA,1612801140,39,25.0213,37.2685,75.27,1.90
525,525,Soledade,0,BR,1612801142,49,-28.8183,-52.5103,78.53,4.50
549,549,Saint-Louis,0,SN,1612801145,21,16.3333,-15.0000,86.00,2.30
550,550,La Orilla,0,MX,1612801146,54,17.9833,-102.2333,79.99,2.19


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# adding new column to hotel_df
hotel_df["Hotel Name"] = ""

/Users/EricShaffer/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
# create the base url for the api search 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}

In [9]:
# make successive api calls for each city and find the nearest hotel within 5000 meters
for index, row in hotel_df.iterrows():
    # get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    #assign the location of the city (lat and lng)
    city_coord = f"{lat}, {lng}"
    
    # add this to the api
    params["location"] = city_coord
    
    # make the api call
    response = requests.get(base_url, params=params).json()
    # getting into the results section of the api     
    results = response['results']
 
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        print(f'The nearest hotel to {row["City"]} is {results[0]["name"]}')
        
    except:
        print("Could not find a hotel within range")
    
    time.sleep(1)

/Users/EricShaffer/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


The nearest hotel to Champerico is Hotel Posada del Mar
The nearest hotel to Castro is Pousada Rota Dos Tropeiros
The nearest hotel to Safaga is Lamar Resort Abu Soma
The nearest hotel to Coyhaique is Entre Cumbres Apart Hotel
The nearest hotel to Ancud is Hotel Balai
The nearest hotel to Jerusalem is King David Hotel
The nearest hotel to Umluj is HP Red Sea Hotel
The nearest hotel to Soledade is Hotel Ágata
Could not find a hotel within range
The nearest hotel to La Orilla is Hotel de Casablanca
The nearest hotel to Podor is La cour du fleuve
The nearest hotel to Lázaro Cárdenas is City Express Lázaro Cárdenas


In [10]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
59,59,Champerico,0,GT,1612801085,66,14.3000,-91.9167,86.00,3.04,Hotel Posada del Mar
83,83,Castro,0,BR,1612801087,51,-24.7911,-50.0119,74.95,4.65,Pousada Rota Dos Tropeiros
124,124,Safaga,0,EG,1612801093,33,26.7292,33.9365,75.20,3.44,Lamar Resort Abu Soma
207,207,Coyhaique,0,CL,1612801103,31,-45.5752,-72.0662,75.20,3.44,Entre Cumbres Apart Hotel
257,257,Ancud,0,CL,1612801109,44,-41.8697,-73.8203,81.00,4.61,Hotel Balai
303,303,Jerusalem,0,IL,1612800974,56,31.7690,35.2163,71.01,3.44,King David Hotel
510,510,Umluj,0,SA,1612801140,39,25.0213,37.2685,75.27,1.90,HP Red Sea Hotel
525,525,Soledade,0,BR,1612801142,49,-28.8183,-52.5103,78.53,4.50,Hotel Ágata
549,549,Saint-Louis,0,SN,1612801145,21,16.3333,-15.0000,86.00,2.30,
550,550,La Orilla,0,MX,1612801146,54,17.9833,-102.2333,79.99,2.19,Hotel de Casablanca


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.symbol_layer(locations, info_box_content=hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))